In [1]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType 
from pyspark.sql import SparkSession

In [5]:
jars = ['./aws-java-sdk-bundle-1.12.262.jar', './hadoop-aws-3.3.4.jar']

In [6]:
spark = (
    # https://medium.com/@abdullahdurrani/working-with-minio-and-spark-8b4729daec6e
    # Set the MinIO access key, secret key, endpoint, and other configurations
    SparkSession
        .builder
        .master("spark://spark:7077")
        .appName('MinIO - Teste')
        .config('spark.hadoop.fs.s3a.endpoint', 'http://minio:9000')
        .config('spark.hadoop.fs.s3a.access.key', 'minioadmin')
        .config('spark.hadoop.fs.s3a.secret.key', 'minioadmin')
        .config('spark.hadoop.fs.s3a.path.style.access', 'true')
        .config('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
        .config('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
        .config('spark.jars', ', '.join(jars))
        # .config('spark.jars.packages', 'com.amazonaws:aws-java-sdk-bundle:1.12.783, org.apache.hadoop:hadoop-aws:3.4.1')
        .getOrCreate()
)

25/05/07 00:17:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
data = [ 
 ("James","","Smith","36636","M",3000),
 ("Michael","Rose","","40288","M",4000),
 ("Robert","","Williams","42114","M",4000),
 ("Maria","Anne","Jones","39192","F",4000),
 ("Jen","Mary","Brown","","F",-1) 
]

schema = (
 StructType([
  StructField("firstname",StringType(),True),
  StructField("middlename",StringType(),True), 
  StructField("lastname",StringType(),True),
  StructField("id", StringType(), True),
  StructField("gender", StringType(), True),
  StructField("salary", IntegerType(), True)
   ])
)

df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



In [8]:
df.show()

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|   id|gender|salary|
+---------+----------+--------+-----+------+------+
|    James|          |   Smith|36636|     M|  3000|
|  Michael|      Rose|        |40288|     M|  4000|
|   Robert|          |Williams|42114|     M|  4000|
|    Maria|      Anne|   Jones|39192|     F|  4000|
|      Jen|      Mary|   Brown|     |     F|    -1|
+---------+----------+--------+-----+------+------+



In [9]:
df.coalesce(1).write.mode('overwrite').parquet('s3a://land/data_2.parquet')

25/05/07 00:17:58 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [10]:
spark.stop()

	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.12.783!aws-java-sdk-bundle.jar (1128948ms)
:: resolution report :: resolve 3626ms :: artifacts dl 1128953ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.783 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   1   |   1   |   0   ||   1   |   1   |
	---------------------------------------------------------------------

:: problems summary ::
:::: WARNINGS
		module not found:  org.apache.hadoop#hadoop-aws;3.4.1

	==== local-m2-cache: tried

	  file:/root/.m2/repository/ org/apache/hadoop/hadoop-aws/3.4.1/hadoop-aws-3.4.1.pom

	  -- artifact  org.apache.hadoop#hadoop-aws;3.4.1!hadoop-aws.jar:

	  file:/root/.m2/repository/ org/apache/hadoo